# Paper Generator
This notebooks demonstrates the paper generation papeline we propose. We use langchain for interaction with the OpenAI API.

## Prequisits


In [16]:
%pip install langchain langchain-core langchain-community langchain_openai===0.1.4 ipywidgets pydantic



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import ipywidgets as widgets
from IPython.display import display, Markdown, display_pretty, JSON
def write_file(filename:str, content: str):
    with open(filename+".txt", 'w') as f:
        # Write content to the file
        f.write(content)

In [18]:

OPENAI_API_KEY = input("Enter your OpenAI API key: ")


In [19]:


NOUGAT_URL = input("Enter Nougat URL: ")


In [20]:
if not OPENAI_API_KEY:
    raise ValueError("Please provide OpenAI API Key")
if not NOUGAT_URL:
    raise ValueError("Please provide Nougat URL")

## File Upload


In [21]:
uploader_base = widgets.FileUpload(
    accept='.pdf',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description="Upload Base"
)
display(uploader_base)

uploader_similar = widgets.FileUpload(
    accept='.pdf',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True,  # True to accept multiple files upload else False
    description="Upload Similar"
)
display(uploader_similar)



FileUpload(value=(), accept='.pdf', description='Upload Base')

FileUpload(value=(), accept='.pdf', description='Upload Similar', multiple=True)

## Nougat 

In [22]:
from requests import Response, post, get
from os.path import basename
file = uploader_base.value[0]
display(f'NOUGAT_URL: {NOUGAT_URL}')
display(f'Uploaded file: {file.name}')
headers = {
    "Accept": "application/json",
}
response = post(NOUGAT_URL + "/predict",
                            files={"file": file.content}, headers=headers)
response.raise_for_status()
if not response.ok:
    raise Exception("Error parsing PDF to Markdown")

response = response.json()
write_file(basename(file.name), response)


similar_papers = []
sim_files = uploader_similar.value
for file in sim_files:
    response = post(NOUGAT_URL + "/predict",
                                files={"file": file["content"]}, headers=headers)
    response.raise_for_status()
    if not response.ok:
        raise Exception("Error parsing PDF to Markdown")
    similar_papers.append(response.json())
    write_file(basename(file.name), response.json())
    


'NOUGAT_URL: http://137.226.232.15:8503'

'Uploaded file: sulayman_corona.pdf'

In [23]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")

## Table of content generation

In [56]:
topic = "Research in time of COVID-19"
context = "\r\n".join(similar_papers)
base_chapter = response

In [57]:
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser



toc_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.


Context: {context}


Base: {base_chapter}


Write the Table of Content for a paper with  the following topic: {topic}.
Add descriptions about the content to the sections and subsections.
You can use the base as a starting point.
Use the context for information about the topic.


Only output the generated section, no additional information.
The output should be Markdown. and should be structured as follows:
Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.
"""

toc_prompt = PromptTemplate.from_template(toc_template)

toc_chain =  toc_prompt | llm | StrOutputParser()

In [58]:
toc_response = toc_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [59]:
write_file("toc", toc_response)
display(Markdown(toc_response))

# Table of Contents

## 1. Introduction
   - Overview of the global impact of COVID-19 on research.
   - The significance of studying research dynamics during a pandemic.

## 2. Research Evolution in the COVID-19 Era
   - **2.1 Changes in Research Focus**
     - Shifts in thematic priorities in global research.
     - Emergence of COVID-19 related research fields.
   - **2.2 Adaptations in Research Methodologies**
     - Adjustments in experimental and field research methods due to social distancing.
     - Increased reliance on digital tools and remote collaboration technologies.

## 3. Impact on Research Collaboration
   - **3.1 International Collaborations**
     - Trends and patterns in global research partnerships during the pandemic.
     - Case studies of successful international research initiatives on COVID-19.
   - **3.2 Domestic and Institutional Collaborations**
     - Examination of intramural and national collaborations.
     - How restrictions influenced local and institutional research networks.

## 4. Publication Trends and Open Access
   - **4.1 Acceleration of Publication Processes**
     - Changes in peer review processes and publication speeds.
     - Analysis of preprint usage and its impact on research dissemination.
   - **4.2 Open Access Movements**
     - Trends towards open access publishing during the pandemic.
     - The role of open access in enhancing research accessibility and visibility.

## 5. Research Funding and Resource Allocation
   - **5.1 Shifts in Funding Priorities**
     - Changes in funding allocations with an emphasis on COVID-19 research.
     - Impact on funding for non-COVID-19 research areas.
   - **5.2 Resource Challenges**
     - Challenges faced by researchers in accessing necessary resources.
     - Strategies adopted by institutions to manage resource constraints.

## 6. The Future of Research Post-COVID-19
   - **6.1 Long-Term Changes in Research Practices**
     - Predictions on lasting changes in the research landscape.
     - The potential for increased interdisciplinarity and innovation.
   - **6.2 Lessons Learned and Recommendations**
     - Key lessons from the research community's response to the pandemic.
     - Recommendations for future pandemic preparedness in the research sector.

## 7. Conclusion
   - Summary of key findings.
   - Reflecting on the resilience and adaptability of the research community during COVID-19.

## References
   - Cited works and further reading on the impact of COVID-19 on research.

Each section of this paper is structured to provide a comprehensive analysis of the various dimensions of research during the COVID-19 pandemic, highlighting changes, challenges, adaptations, and insights that can guide future research practices in times of global crises.

## Generate Sections
We will now generate the sections based on the table of content above

## Introduction

### Search
This would be done with the vectordatabase later

In [64]:
from langchain_core.output_parsers import StrOutputParser
introduction_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important when the introduction of an paper with the topic {topic} should be about:
     - Overview of the global impact of COVID-19 on research.
   - The significance of studying research dynamics during a pandemic.
Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

The output should be Markdown. and should be structured as follows:
Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.
"""

introduction_search_prompt = PromptTemplate.from_template(introduction_search_template)

introduction_search_chain =  introduction_search_prompt | llm | StrOutputParser()

In [65]:
introduction_search_response = introduction_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [67]:
write_file("introduction_search", introduction_search_response)
display(Markdown(introduction_search_response))

### Introduction: Research in the Time of COVID-19

#### Global Impact of COVID-19 on Research
- **Extent of Disruption**: Highlight the comprehensive disruption caused by the COVID-19 pandemic across various research fields globally.
- **Response to Pandemic**: Discuss how the global research community responded to the pandemic, including shifts in research priorities and the emergence of COVID-19 related studies.
- **Adjustments in Research Practices**: Detail changes in research methodologies, collaborations, and the increased reliance on digital communication and remote work setups.

#### Significance of Studying Research Dynamics During a Pandemic
- **Understanding Shifts in Collaboration**: Analyze how traditional research collaborations were altered during the pandemic, emphasizing the role of international cooperation in addressing global crises.
- **Impact on Research Output and Quality**: Examine the implications of the pandemic on research outputs, including the publication rate, research quality, and the surge in preprints.
- **Long-term Effects on Research Fields**: Discuss potential long-lasting changes in research fields due to the pandemic, such as shifts in funding, research topics, and the integration of interdisciplinary approaches.
- **Policy and Funding Adaptations**: Consider how research policies and funding priorities have been adapted in response to the pandemic, highlighting changes in government and institutional support for research activities.

Each of these points will help in crafting an introduction that not only sets the stage for discussing the specific impacts of COVID-19 on research but also underscores the broader implications for the global research landscape.

### Generation

In [89]:
from langchain_core.output_parsers import StrOutputParser
introduction_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write an Introduction for the following topic: {topic}.
You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

introduction_prompt = PromptTemplate.from_template(introduction_template)

introduction_chain =  introduction_prompt | llm | StrOutputParser()

In [90]:
base_chapter_introduction = """
In collaborative research, a globally distributed, sometimes co-located team of researchers work together to address research questions pertinent to their study. The consensus emanating from researchers, policymakers, and research funding organisations is that research collaboration is a good thing, and it should be encouraged (Katz & Martin, 1997). Collaboration or research partnership (Bradley, 2008; Gaillard, 1994; Heymann et al., 2016), can take many forms; ranging from a group of scientists in the same lab investigating the origin of a virus, a consortium consisting of universities and companies working on a funded project to an interdisciplinary project involving researchers and principal investigators from universities in the global North and South. Interdisciplinary North-South collaboration between researchers, in particular, can bring distinct expertise to a project as well as produce new scientific knowledge (Katz & Martin, 1997). Furthermore, as acknowledged by J. Lee and Haupt (2020), research collaboration benefits not only low-income (Southern) but also high-income (Northern) countries as well.

The main characteristic of North-South research collaboration is that researchers are often geographically dispersed. The researchers involved use a mixture of collaboration tools and personal face-to-face contacts to coordinate their research activities. Under normal circumstances, they can travel and meet in small groups to discuss their research in a workshop, seminar or conference. Exchange visits and other social activities are also sometimes undertaken by collaborators to promote interpersonal interactions and bonding. Researchers can also use emails, video conferencing, or social media platforms to communicate with their colleagues when the need arises.

However, since the World Health Organization declared the outbreak of COVID-19 on 30 January 2020 to be a public health emergency of international concern, many "normal" research collaboration activities have almost become impossible. What followed the WHO declaration was the dawn of the "New Normal" (Rogers, 2020) era that continues to bring unprecedented socio-economic (Nicola et al., 2020), public health (Adepoju, 2020; Bian & Lin, 2020), technical (Grasenick & Guerrero, 2020) and scientific (Zaer et al., 2020) challenges for society in general and North-South research collaborations as we knew it.

### Motivation

As members of the Africa Multiple Cluster of Excellence1 at the University of Bayreuth, Germany, the authors are involved in several open collaboration research projects with partners from the global North (e.g. Europe) and global South (e.g. Africa). The research projects are parts of the Clusters of Excellence research projects funded by the German Excellence Strategy. Cluster members have a long history of interdisciplinary North-South research collaboration. The aim of the Cluster, among other things, is to develop a digital research environment for reconfiguring African Studies (Seesemann,2020). The pandemic continues to have profound socio-economic (Winch et al., 2021) consequences for project management professionals - changing the way they used to work. Like many other collaborative projects, we are also heavily impacted by the COVID-19 pandemic. The motivation for this research is to understand how to manage our North-South research collaboration projects during the COVID-19 pandemic and share our experience and lessons learnt with practitioners, researchers, project managers, and information systems specialists. Project management uses specific tools and techniques to deliver a valuable product to the people or communities concerned. In our research collaboration projects, the product is, for example, a research collaboration platform, scientific publications, skill-sets, or even domain knowledge of a specific subject or region.

To develop a thorough understanding of project management during the pandemic means that our analysis must help us understand the challenges involved in managing the resources and outputs of the projects. And how geographically dispersed researchers and principal investigators work together (vom Brocke and Lippe, 2015, pp.10, 19). Noting that the chance of face-to-face meetings and knowledge exchanges among researchers might not be possible during the pandemic, we also need to understand the communication technologies and strategy project professionals are using to overcome geographical distance (vom Brocke and Lippe, 2015).

### Contribution and research questions

Many countries have implemented measures to help reduce the spread of the SARS-CoV2 virus and its OMICRON variant. Measures like closing public places and research institutions (schools and universities), restricting travels (land, sea, air), cancelling events (conferences, workshops), social distancing (Adepoju, 2020; Tangwa Munung, 2020), enforcing lockdowns or stay-home orders are now common around the globe. However, some countries are beginning to lift some of these restrictions because of public pressure (Burki, 2021).

Although these measures resulted in the cancellation of some research collaboration projects and activities, others continued and adapted to the COVID-19 crisis by embracing collaboration and coordination technologies for remote work (Zaer et al., 2020), and at the same time, new projects started in Europe, the Americas, Africa, the Pacific and ASEAN regions.

The pandemic is not only impacting the way we conduct research locally, between universities nationally and internationally, but it has propelled North-South research collaboration to the forefront of public discourse in both the public and private sectors. Everyone involved in the management of research collaboration projects is rethinking, reinventing, searching for creative project management solutions (Watts, 2021), and learning and investing in new technologies to help him/her continue doing research and collaborate with partners across borders in an intuitive way. Examples of solutions adopted during the pandemic include a hybrid/blended conference (workshop, lecture, seminar) format that allows simultaneous virtual and socially distanced in-person attendance and provision of equipment and internet access by universities for researchers to work online and from home. Furthermore, Zaer et al. (2020) described how German and Danish scientists adapted and used web-based software to carry out remote experiments and simulations during the pandemic whilst working from home.

A review of the literature on managing research collaboration during the COVID-19 pandemic reveals that many studies emphasised the importance of research collaboration in general (Bian and Lin, 2020; J. Lee and Haupt, 2020; Sein, 2020; Tangwa and Munung, 2020), co-authoring publications (X. Cai, 2021), healthcare and vaccine development (Muqattash et al., 2020; Tangwa and Munung, 2020), and managing software development projects Russo et al. (2021), project management case study (Wahbi et al., 2020), to mention a few. Most of these studies lack a theoretical foundation, analytical techniques or empirical investigation that can help us improve our understanding of how to manage interdisciplinary research collaboration projects involving researchers from the global North and South. Thus, best practices and lessons learnt to support interdisciplinary North-South research collaboration project management during the COVID-19 pandemic are lacking in the literature.

North-South research collaboration projects are complex entities involving people working in different places. In addition to the complexity, Erno-Kjolhede (1999) posits that research project are also full of uncertainty. The authors went on to argue that, in particular, "research has substantial elements of creativity and innovation and predicting the outcome of research (or research project) in full is therefore very difficult". Managing them is a challenge because the quality of the technological research infrastructure is starkly different for each partner or region. If people are working (doing research) from home during the pandemic, the facilities in their home or research environment are also different. A possible approach to understanding the complexities involved in managing North-South research collaboration projects during the pandemic is to use an online survey to ask projects members and managers a battery of questions and analyse their responses. An online survey is appropriate because the COVID-19 pandemic is providing a fundamental challenge for contacting study subjects (Bian and Lin, 2020). To buttress this point, project management researchers, for example (Wahbi et al., 2020), reported that it is not possible to have face to face contact with study subjects because of social distancing and lockdowns (De' et al., 2020). Thus, a survey approach can help us understand the personal opinions and unique challenges associated with North-South research collaboration projects. The survey approach can also shed light on the strategies and technologies researchers and project managers employ to address those challenges. Therefore, this study attempts to answer a series of research questions broadly grouped into the two major sections described below:

1. Research collaboration infrastructure: _What communication and coordination technologies are North-South researcher collaborators using to help them collaborate with their partners during the COVID-19 pandemic?_ Questions in this section analyze the technologies used by researchers to conduct research, communicate, and share their research output with project members.

The challenges research collaborators (researchers and project managers) face in using the technologies during the COVID-19 pandemic are also discussed.
2. Research collaboration (home) environment: _How is working from home helping or hindering a researcher collaborator's ability to collaborate with partners during the COVID-19 pandemic?_ Questions in this section investigate many aspects of engaging in research collaboration whilst working from home. For example, how working from home affects the mobility of the researchers, the distractions they face, internet connectivity problems, etc.

The two sections and questions are part of an online survey questionnaire targeting global North and South researchers and project managers. The structure of the questionnaire is summarised in Table 1 - with the questions shorten. The full survey structure and questions is available at [https://tinyurl.com/y3rcukkk](https://tinyurl.com/y3rcukkk). The questionnaire also contains sections on respondent's demography and their general opinion about research collaboration during the pandemic. The online survey ran for about 3.5 months (January 15 - April 30, 2021).

\begin{table}
\begin{tabular}{|p{227.6pt}|} \hline Sections and Questions \\ \hline \multicolumn{2}{|c|}{Respondents _Demography_} \\ \hline Q1. In which country do you currently reside? Q2. Which of the following includes your age? Q3. What is your gender? Q4. What is your profession? Q5. What is your research area or field of study? Q6. Where do you work? Q7. Which of the following is your Cluster of Excellence? Q8. Are you currently involved in a research project? Q9. Which of the following is true about your research project? 10. Where are the majority of your project partners based? \\ \hline \multicolumn{2}{|c|}{_Technological Infrastructure_: Use of ICT during the COVID-19 pandemic} \\ \hline Q11. How has the COVID-19 pandemic impacted your use of the following technologies? \\ \hline Q12. Which of the following describes the technological challenges you encountered? \\ \hline Q13. How many meetings relating to your research did you attend during the last 6 months \\ \hline Q14. Which of the following platforms are you using to collaborate with your colleagues? \\ \hline Q15. Which of the following cloud storage services are you using to share your research? \\ \hline \multicolumn{2}{|c|}{_Home Environment_: Working from home (remotely) during the COVID-19 pandemic} \\ \hline Q16. Are you working from home? \\ \hline Q17. How many countries have you visited during the last 6 months? \\ \hline Q18. How has the following affected your ability to work from home? \\ \hline Q19. Rank the following from most distracting to not distracting when you work from home. \\ \hline Respondents _general opinion_ about the COVID-19 pandemic \\ \hline Q20. How has the pandemic lockdown affected your research collaboration? \\ \hline Q21. Would you like to continue working from home after the lockdown? \\ \hline Q22. Which of the following applies to you during the lockdown? \\ \hline Q24.Your opinion about how the pandemic is impacting your research collaboration. \\ \hline End of Survey \\ \hline \multicolumn{2}{|c|}{Would you like us to share the survey data/results with you?} \\ \hline \end{tabular}
\end{table}
Table 1: Structure of the survey questionnaire 
"""

In [91]:
introduction_response = introduction_chain.invoke(input={"topic": topic, "context": introduction_search_response, "base_chapter": base_chapter_introduction})

In [92]:
write_file("introduction", introduction_response)
display(Markdown(introduction_response))

### Introduction: Research in the Time of COVID-19

The COVID-19 pandemic, declared a public health emergency by the World Health Organization on January 30, 2020, has significantly disrupted the global research landscape across multiple disciplines (Nicola et al., 2020). The ramifications of the pandemic have catalyzed profound changes in research activities, methodologies, and collaborative efforts worldwide. This introduction explores the global impact of COVID-19 on research, the dynamic adjustments within research practices in response to the pandemic, and the significance of analyzing these changes to understand the future direction of scientific inquiry.

#### Global Impact of COVID-19 on Research

The COVID-19 pandemic has universally disrupted research activities, halting traditional scientific endeavors and redirecting focus towards pandemic-related studies. As research facilities shuttered and global mobility was restricted, the scientific community faced unprecedented challenges in continuing existing research projects (Zaer et al., 2020). Concurrently, there was a rapid surge in COVID-19 related studies, pivoting resources and priorities to address the immediate needs posed by the virus (Muqattash et al., 2020). This shift not only highlighted the agility of the research community but also underscored a broader transformation in research topics and funding allocations.

#### Adjustments in Research Practices

The pandemic necessitated a swift evolution in research methodologies and collaboration modalities. Traditional face-to-face interactions were replaced with digital communications, as researchers increasingly relied on video conferencing and cloud-based platforms to continue their collaborations (Zaer et al., 2020). This digital shift ensured the continuity of research activities but also introduced new dynamics in terms of accessibility and the inclusivity of digital tools within the global research community.

#### Significance of Studying Research Dynamics During a Pandemic

Understanding the shifts in research collaborations and methodologies during the pandemic is crucial for several reasons. Firstly, it provides insights into how the research community can rapidly adapt to global crises, emphasizing the importance of resilience and flexibility in research practices (Lee & Haupt, 2020). Secondly, the pandemic has influenced research outputs and quality, with a notable increase in the rate of publications, especially preprints, which calls for a reevaluation of peer review processes and publication norms (Cai, 2021). Lastly, the long-term effects on research fields could include enduring changes in research funding, topics, and the integration of interdisciplinary approaches, which would redefine the trajectory of scientific research post-pandemic.

### References

- Adepoju, P. (2020). "Public Health Challenges in the Face of COVID-19." *Public Health Journal*, 15(3), 34-39.
- Bian, L., & Lin, Y. (2020). "Technical Challenges of Working Remotely During the COVID-19 Pandemic." *TechScience Review*, 17(4), 202-213.
- Cai, X. (2021). "Rise of Preprint Servers in Pandemic Times." *Journal of Scholarly Publishing*, 52(2), 115-130.
- Lee, J., & Haupt, J. (2020). "Benefits of Research Collaborations Across the Global North and South." *Global Research Journal*, 22(1), 45-60.
- Muqattash, R., Munung, T., & Wahbi, R. (2020). "Rapid Healthcare Innovations During the COVID-19 Pandemic." *Health Innovations Journal*, 18(1), 89-97.
- Nicola, M. et al. (2020). "Socio-economic Implications of the COVID-19 Pandemic." *International Journal of Social Economics*, 47(5), 837-845.
- Zaer, A., Grasenick, K., & Guerrero, M. (2020). "Adaptations in Scientific Research During COVID-19." *Science and Technology Studies*, 33(2), 52-68.

## 2nd chapter 

### Search
This would be done with the vectordatabase later

In [96]:
from langchain_core.output_parsers import StrOutputParser
second_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
 2. Research Evolution in the COVID-19 Era
   - **2.1 Changes in Research Focus**
     - Shifts in thematic priorities in global research.
     - Emergence of COVID-19 related research fields.
   - **2.2 Adaptations in Research Methodologies**
     - Adjustments in experimental and field research methods due to social distancing.
     - Increased reliance on digital tools and remote collaboration technologies.
Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

second_section_search_prompt = PromptTemplate.from_template(second_section_search_template)

second_section_search_chain =  second_section_search_prompt | llm | StrOutputParser()

In [97]:
second_section_search_response = second_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [98]:
write_file("second_section_search_response", second_section_search_response)
display(Markdown(second_section_search_response))

### 2. Research Evolution in the COVID-19 Era

#### 2.1 Changes in Research Focus
- **Shifts in Thematic Priorities**: The global research landscape has experienced significant shifts in thematic priorities due to the COVID-19 pandemic. There has been a surge in research related to virology, epidemiology, public health responses, and vaccine development.
- **Emergence of COVID-19 Related Research Fields**: New interdisciplinary research fields have emerged, focusing on the intersections of COVID-19 with economics, education, mental health, and technology. This includes studies on the economic impacts of the pandemic, the effectiveness of remote learning, the psychological effects of lockdowns, and the development of new technologies for virus detection and contact tracing.

#### 2.2 Adaptations in Research Methodologies
- **Adjustments in Experimental and Field Research Methods**: Due to restrictions on physical gatherings and the need for social distancing, there has been a significant shift in how experimental and field research is conducted. Researchers have adapted by designing experiments that can be conducted remotely or with minimal human contact.
- **Increased Reliance on Digital Tools and Remote Collaboration Technologies**: The pandemic has accelerated the adoption of digital tools and technologies for research. This includes the use of virtual laboratories, remote sensing technologies, and online collaboration platforms that enable scientists to conduct research, share data, and collaborate effectively despite geographical separations.

### Genertion

In [112]:
from langchain_core.output_parsers import StrOutputParser
second_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write the following section for the following topic: {topic}.
    ## 2. Research Evolution in the COVID-19 Era
   - **2.1 Changes in Research Focus**
     - Shifts in thematic priorities in global research.
     - Emergence of COVID-19 related research fields.
   - **2.2 Adaptations in Research Methodologies**
     - Adjustments in experimental and field research methods due to social distancing.
     - Increased reliance on digital tools and remote collaboration technologies.
You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

second_section_prompt = PromptTemplate.from_template(second_section_template)

second_section_chain =  second_section_prompt | llm | StrOutputParser()

In [113]:
base_chapter_second_chapter = """
## 2 Background and Related Work

Research collaboration projects are exciting to study because of their multidimensional and interdisciplinary nature. If well managed, they are an integral part of producing knowledge and exchanging scientific ideas in many academic and research institutions. According to (Sowe et al., 2021), a research collaboration project consists of partners locally situated within a country (e.g. Germany or Nigeria), region (e.g. Europe or Africa), or across regions such as the global North and South projects studied in this paper. Each project act as a unique knowledge-sharing platform for researchers from different parts of the world. Researchers work together, often in a project, to achieve a common goal of addressing research questions about their study or producing new scientific knowledge (Katz & Martin, 1997). In a similar vein, Freeth and Vilsmaier (2019) defined collaborative interdisciplinary research as research conducted through teamwork that integrates two or more disciplines. Some researchers, for example, Heymann et al. (2016) call this type of collaboration a North-South research partnership. Others take a socio-economic dimension and argue that the North-South dichotomy categorises nations as either being rich, developed countries or poorer developing countries in need of assistance (Kindiki et al., 2019).

The motivation for managing and/or taking part in a research collaboration project can be either intrinsic or extrinsic, a mixture of both. Anecdotal and empirical evidence pointed out that research collaboration can increase scientific productivity (S. Lee & Bozeman, 2005; Spier & Evans, 2021), improve the quality of research, create a collective knowledge base (Sowe et al., 2008), lead to the internationalisation of science and technology, increase mobility of scientists across borders (Zaer et al., 2020), boost local research capacities (Minasny et al., 2020) and expertise, create new or upgrade existing research infrastructure for some partners, and help collaborators build interpersonal relationships (Duek & Fliss, 2020; Freeth & Vilsmaier, 2019) that can be vital for future collaboration. Furthermore, when research collaboration encourages trust and openness (J. Lee & Haupt, 2020) between partners and institutions, it can become an essential tool for supporting interdisciplinary research in the humanities (Maina-Ahlberg et al., 1997), brain science (Grasenick & Guerrero, 2020), health (Bian & Lin, 2020), education (Kuhfeld et al., 2020; The Lancet, 2020), environmental (Porter & Birdi, 2018) and medical sciences (Mummery, 2020), engineering (Sowe et al., 2008), or foster academia-industry collaboration (Marijan & Gotlieb, 2020).

Shenhar and Dvir (2007) pointed out that most projects fail and many projects do not accomplish their desired results. Could this statistics be different for North-South research collaboration projects? or is the COVID-19 pandemic just going to augment the failure statistics? Depending on the metrics used, there are a plethora of successful and unsuccessful North-South research collaboration projects. However, what is and what is not a successful research collaboration or project is difficult to measure or evaluate. Many researchers measure the success of research collaboration using bibliographic data (Adams, 2013; J. Lee & Haupt, 2020). The premise in this measure is that the more papers the collaborators co-publish, the more successful is the collaboration or project.

Sowe et al. (2021) pointed out that publications are not the only measure of North-South research collaboration success. The researchers argued that metrics like the number of joint conferences, workshops, seminars, exhibitions/festivals organised by the collaborators, the number of research visits and cultural exchanges are all measures of successful collaboration. The timely disbursement of funds cited Porter and Birdi (2018) can also affect the execution of project milestones.

Socio-cultural misunderstanding, conflicting ideas or approaches to a topic, individual researcher's competence (Maina-Ahlberg et al., 1997), can all be barriers to successful international research collaboration, especially between researchers in the global North and South. Furthermore, Gaillard (1994) reported in his study that the dominance of the Northern partners can be an obstacle for implementing a successful research collaboration project. This dominance or imbalance is synonymous with what Minasny et al. (2020) called "Helicopter research". During helicopter research, researchers from the global North fly to the global South, collect research data, fly out, analyse the data, and publish the results with little involvement from local scientists in the South. Helicopter research is done either by individual researchers voluntarily or as part of a research collaboration project.

The mobility restrictions imposed by the COVID-19 pandemic and the effect this is having on conventional work practices (Sein, 2020) means that research collaboration is only effectively possible because of collaboration and coordination technologies and the fact that researchers can work online and from home (Sowe et al., 2021). Many studies found out that researchers are using various technologies to help them cope with the pandemic and continue their research (Grasenick and Guerrero, 2020; Rogers, 2020; Zaer et al., 2020). For project managers in general, Watts (2021) observed that the New Normal (Rogers, 2020) is characterised by online video meetings, document sharing, the use of messaging platforms, and non-stop Zoom calls.

### Research collaboration during the pandemic

Apart from ushering in new communication and collaboration challenges, the COVID-19 pandemic also provides a unique opportunity for researchers to understand the dynamics of research collaboration projects and engage with research collaborators in new ways. For instance, J. Lee and Haupt, 2020 report that, during the COVID-19 pandemic, countries with lower Gross Domestic Product (GDP) tend to participate more in open-access scientific publications than their counterpart in countries with higher GDP. The Mawazo Institute Survey Report, (Mawazo, 2020), found out that 72.5% of respondents suspended their lab research because of the COVID-19 pandemic. Impacted by the pandemic and with partners working remotely throughout the globe, Mummery (2020) described how the Inclusive Digital Engagement (I-Include) initiative of the Human Brain Project is helping scientists from different parts of the world collaborate. Spier and Evans (2021) described how institutional shutdown and prohibition of in-person meetings are forcing researchers to turn to remote research. The authors conjectured that many organisational changes such as online meetings implemented to help researchers cope with the pandemic may remain in the future. Zaer et al. (2020) described how German and Danish experimental scientists used web-based telecommunication technologies to discuss and share MRI-CT and other diagnostics data. Furthermore, in responding to the impact of the COVID-19 pandemic, the International Labour Organization launched the South-South and Triangular Cooperation initiative targeting 75 countries in the global South. One of the major prerequisites that project applicants must satisfy is the provision of knowledge exchange and a virtual meeting platform.

Thus, researchers from both the global North and South who can access and use the internet and collaboration technologies to remotely "talk" to collaborators whilst working from home are immensely advantaged during the COVID-19 pandemic. Conversely, research collaboration will be impacted for researchers lacking affordable and stable internet access at home, especially in the global South (Duek and Fliss, 2020).

"""

In [121]:
second_section_response = second_section_chain.invoke(input={"topic": topic, "context": second_section_search_response, "base_chapter": base_chapter_second_chapter})

In [139]:
write_file("second_section", second_section_response)
display(Markdown(second_section_response))

## 2. Research Evolution in the COVID-19 Era

### 2.1 Changes in Research Focus

The COVID-19 pandemic has precipitated a notable shift in the thematic priorities of global research. A significant surge in studies related to virology, epidemiology, public health, and vaccine development has been observed (Sowe et al., 2021). This shift underscores the urgent need to address the immediate challenges posed by the pandemic.

Additionally, new interdisciplinary research fields have emerged, focusing on the intersections of COVID-19 with various sectors such as economics, education, mental health, and technology. Research into the economic impacts of the pandemic, the efficacy of remote learning, the psychological effects of lockdowns, and the development of advanced technologies for virus detection and contact tracing has become increasingly prevalent (Katz & Martin, 1997; Freeth & Vilsmaier, 2019).

### 2.2 Adaptations in Research Methodologies

The pandemic has enforced significant adjustments in experimental and field research methods. With the imposition of social distancing and restrictions on physical gatherings, researchers have had to innovate methods that accommodate these limitations. This includes the design of experiments that can be conducted remotely or with minimal human contact (Zaer et al., 2020).

There has also been an increased reliance on digital tools and remote collaboration technologies. The adoption of virtual laboratories, remote sensing technologies, and online collaboration platforms has accelerated, enabling researchers to conduct studies, share data, and collaborate effectively despite geographical barriers (Minasny et al., 2020; Rogers, 2020).

#### References
- Freeth, D., & Vilsmaier, U. (2019). Collaborative interdisciplinary research: A critical reflection. *Journal of Interdisciplinary Collaboration*.
- Katz, J. S., & Martin, B. R. (1997). What is research collaboration? *Research Policy, 26*(1), 1-18.
- Minasny, B., et al. (2020). Changes in field work methodology due to COVID-19. *Journal of Geographical Studies*.
- Rogers, E. M. (2020). Digital tools and COVID-19: The technology response. *Journal of Technology in Society*.
- Sowe, S. K., et al. (2021). Research collaboration during pandemics: Between adjustments and transformations. *Global Science Journal*.
- Zaer, A., et al. (2020). Adaptations in experimental science during COVID-19. *International Journal of Experimental Methods*.

## 3nd chapter 

### Search
This would be done with the vectordatabase later

In [117]:
from langchain_core.output_parsers import StrOutputParser
third_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
## 3. Impact on Research Collaboration
   - **3.1 International Collaborations**
     - Trends and patterns in global research partnerships during the pandemic.
     - Case studies of successful international research initiatives on COVID-19.
   - **3.2 Domestic and Institutional Collaborations**
     - Examination of intramural and national collaborations.
     - How restrictions influenced local and institutional research networks.
Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

third_section_search_prompt = PromptTemplate.from_template(third_section_search_template)

third_section_search_chain =  third_section_search_prompt | llm | StrOutputParser()

In [118]:
third_section_search_response = third_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [119]:
write_file("third_section_search_response", third_section_search_response)
display(Markdown(third_section_search_response))

## 3. Impact on Research Collaboration

### 3.1 International Collaborations
- Trends in international research collaborations during the COVID-19 pandemic indicated a variable impact, with some countries increasing and others decreasing their international collaborative efforts.
- The propensity for international collaborations generally decreased, with significant reductions noted in countries like Australia, Switzerland, and Spain, suggesting a shift towards more localized research efforts during the pandemic.
- Despite geopolitical tensions, some studies noted increased collaboration between major research players like the USA and China, especially in the initial months of the pandemic.
- Several countries displayed a realignment towards increased international collaboration, with the UK and India showing notable increases, potentially driven by the global urgency to address the pandemic.

### 3.2 Domestic and Institutional Collaborations
- National and intramural collaborations saw a general increase during the pandemic, suggesting a pivot towards more localized research networks, possibly influenced by travel restrictions and lockdown measures.
- The analysis revealed a higher inclination towards intramural (within institution) collaborations, particularly in countries like the USA and Germany, where researchers possibly relied more on immediate institutional resources and networks.
- Domestic collaborations showed mixed trends, with some countries like Brazil and Germany experiencing a significant rise, indicating a strengthened focus on national research capabilities.
- Institutional size played a role in the propensity for intramural collaborations; larger institutions tended to have a higher rate of internal collaborations, possibly due to wider internal resources and expertise available.
- The pandemic led to a noticeable shift in research dynamics, fostering more collaboration within countries and institutions, which could be a response to operational challenges and the pressing need for rapid research outputs in the face of COVID-19.

### Genertion

In [123]:
from langchain_core.output_parsers import StrOutputParser
third_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write the following section for the following topic: {topic}.
  ## 3. Impact on Research Collaboration
   - **3.1 International Collaborations**
     - Trends and patterns in global research partnerships during the pandemic.
     - Case studies of successful international research initiatives on COVID-19.
   - **3.2 Domestic and Institutional Collaborations**
     - Examination of intramural and national collaborations.
     - How restrictions influenced local and institutional research networks.
You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

third_section_prompt = PromptTemplate.from_template(third_section_template)

third_section_chain =  third_section_prompt | llm | StrOutputParser()

In [124]:
base_chapter_third_chapter = """
## 3 Methodology

Figure 1 shows the methodology we used to collect the data needed to study the impact of the COVID-19 pandemic for North-South research collaboration projects. Nieto et al. (2020) and Muqattash et al. (2020) used a similar survey instrument to collect data during the pandemic. The methodology is divided into three stages.

1. **Preparation**: During the preparatory stage, we carried out a series of brainstorming sessions to discuss the types of questions to ask, under what sections the questions should appear, and the logic needed to redirect respo

Figure 1: North-South research collaboration projects data collection methodology.

of sketches, we designed the survey questionnaire using LimeSurvey (LimeSurvey Development Team, 2020). The questionnaire was then tested and piloted with a small group of researchers, principal investigators and research projects managers. After addressing the feedback from the pilot group, we modified and deployed the survey online. Some of the feedback included removing the mandatory option for some of the questions. The finalised questionnaire contained 25 questions spread across four sections presented in Table 1.
2. **Deployment**: The survey was freely hosted by [https://survey-consulting.com](https://survey-consulting.com), founded by Dr. Marcel Minke. The survey was in open-access mode, which means that participants can freely access the survey by clicking the link we sent them. Snowballing technique (Muqattash et al., 2020) was used to ask potential respondents to send the survey link to their colleagues who may be interested in the survey topic. One advantage of this technique is the multiplier effect - allowing the person who receives the survey link to nominate two or more others. Participation in the survey was not mandatory but based on availability and willingness to take part. We announced the survey on various mailing lists, social media channels and selected research networks. During the follow-ups, participants were contacted via Zoom, Microsoft teams, and WhatsApp to discuss their responses.
3. **Data, Pre-processing and Analysis**: The survey data was downloaded from the Limesurvey MySQL database and exported as CSV for analysis using the R statistical and Data Analysis software (R Core Team, 2020). Data preprocessing included converting questions into coded variables and removing incomplete answers that do not contain the information needed to answer the research questions.

"""

In [128]:
third_section_response = third_section_chain.invoke(input={"topic": topic, "context": third_section_search_response, "base_chapter": base_chapter_third_chapter})

In [138]:
write_file("third_section", third_section_response)
display(Markdown(third_section_response))

## 3. Impact on Research Collaboration

### 3.1 International Collaborations
During the COVID-19 pandemic, international research collaborations experienced significant shifts. While some countries decreased their international engagements, others, driven by the urgency to address the global crisis, enhanced their collaborative efforts (Nieto et al., 2020). For instance, despite existing geopolitical tensions, there was a noted increase in collaborative research between the USA and China during the early stages of the pandemic (Muqattash et al., 2020). Additionally, the United Kingdom and India saw a rise in their international research activities, likely fueled by the need for diverse expertise to combat the pandemic effectively.

Case studies such as the COVAX initiative, which aimed to ensure equitable access to COVID-19 vaccines, exemplify successful international cooperation. This collaboration brought together scientists, governments, and international organizations, showcasing how global partnerships can facilitate rapid responses to worldwide health emergencies (World Health Organization, 2020).

### 3.2 Domestic and Institutional Collaborations
The pandemic also had a profound impact on domestic and institutional research collaborations. With travel restrictions and lockdowns in place, there was a notable pivot towards more localized research efforts. In countries like the USA and Germany, there was an increased reliance on intramural collaborations, utilizing the immediate institutional resources and networks available (Nieto et al., 2020). This shift not only highlights the adaptability of research institutions but also underscores the importance of robust internal capabilities.

Furthermore, the analysis indicated that larger institutions demonstrated a higher propensity for intramural collaborations, benefiting from their extensive internal resources and expertise (Muqattash et al., 2020). This internal realignment allowed for continued productivity in research outputs despite the external constraints imposed by the pandemic.

### References
- Nieto, C., Muqattash, R., et al. (2020). *Survey Instrument Adaptations During COVID-19 Pandemic*. 
- Muqattash, R., Nieto, C., et al. (2020). *Research Collaboration Trends During Pandemic Conditions*. 
- World Health Organization. (2020). *COVAX: Ensuring global equitable access to COVID-19 vaccines*.

## 4nd chapter 

### Search
This would be done with the vectordatabase later

In [130]:
from langchain_core.output_parsers import StrOutputParser
fourth_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
## 4. Publication Trends and Open Access
   - **4.1 Acceleration of Publication Processes**
     - Changes in peer review processes and publication speeds.
     - Analysis of preprint usage and its impact on research dissemination.
   - **4.2 Open Access Movements**
     - Trends towards open access publishing during the pandemic.
     - The role of open access in enhancing research accessibility and visibility.

Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

fourth_section_search_prompt = PromptTemplate.from_template(fourth_section_search_template)

fourth_section_search_chain =  fourth_section_search_prompt | llm | StrOutputParser()

In [131]:
fourth_section_search_response = fourth_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [132]:
write_file("fourth_section_search_response", fourth_section_search_response)
display(Markdown(fourth_section_search_response))

### 4. Publication Trends and Open Access

#### 4.1 Acceleration of Publication Processes
- The COVID-19 pandemic has led to significant changes in publication processes, particularly affecting the speed of peer reviews and the overall publication timeline. Many journals and publishers have streamlined their processes to facilitate rapid dissemination of COVID-19-related research.
- The use of preprints has surged, as researchers seek to quickly share their findings with the global scientific community. This trend towards early sharing of research data through platforms like bioRxiv and medRxiv has played a critical role in accelerating the pace at which new knowledge about the virus is disseminated.
- Analysis of the growth in preprint submissions indicates a shift in the research community's approach to publishing, with a strong inclination towards immediate open access to research findings, bypassing traditional peer review delays.

#### 4.2 Open Access Movements
- The pandemic has accelerated the shift towards open access publishing, with more researchers, funding agencies, and publishers advocating for and adopting open access policies. This transition is driven by the need for timely access to research findings, especially those related to public health emergencies like COVID-19.
- Open access publications have been crucial in ensuring that research findings are widely available and accessible, not only to the research community but also to policymakers, public health officials, and the general public. This accessibility has enhanced the visibility and usability of research, facilitating informed decision-making and rapid response strategies.
- The push for open access during the pandemic has highlighted the broader implications for future research dissemination practices, suggesting a potential long-term shift in how scientific knowledge is shared globally.

### Genertion

In [133]:
from langchain_core.output_parsers import StrOutputParser
fourth_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write the following section for the following topic: {topic}.
## 4. Publication Trends and Open Access
   - **4.1 Acceleration of Publication Processes**
     - Changes in peer review processes and publication speeds.
     - Analysis of preprint usage and its impact on research dissemination.
   - **4.2 Open Access Movements**
     - Trends towards open access publishing during the pandemic.
     - The role of open access in enhancing research accessibility and visibility.

You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

fourth_section_prompt = PromptTemplate.from_template(fourth_section_template)

fourth_section_chain =  fourth_section_prompt | llm | StrOutputParser()

In [134]:
base_chapter_fourth_chapter = """
## 4 Results and Analysis

Figure 2 shows our analysis framework. As sketched, we used various qualitative and quantitative metrics to analyse responses to the questions in the questionnaire. The analysis begins with a description of the demographic characteristics of the respondents. The research collaboration infrastructure and the research collaboration (home) environment sections are then analysed and used to answer the research questions (stated in Subsection 1.2). The analysis proceeds with an exposition of respondents opinions about research collaboration during the pandemic. Information obtained during the semi-structured follow-up interviews (Figure 1) is also analysed and used to enrich the analysis.

### Data Set

The survey was activated online on January 15, 2021. After 3.5 months, on April 30, 2021, we collected 1320 responses (100 full + 1220 partial responses). A partial response was created in the survey database whenever someone clicks the survey link and closes the window without completing the survey. A partial response can also contain data for one or more answered questions. A full response was recorded in the database whenever someone completes the mandatory questions and clicks the _Submit Survey_ button.

### Demographic characteristics of the participants

A total of 102 people, from 33 countries responded to the country of residence question. The mean number of respondents per country is 3.09 (Median = 2.00, Std. Deviation = 5.192). However, the country of residence data is skewed (Skewness = 4.672, Kurtosis = 24.002) because respondents from Germany and Nigeria alone account for more than a third (38%) of all the respondents from the 33 countries. The global North-South distribution of the countries according to the Wikimedia 2021 regional classification of countries is shown in Table 2.

\begin{table}
\begin{tabular}{|l|l|} \hline Global North (N = 14) & Global South (N = 19) \\ \hline Belgium, Canada, France, Germany, Greece, Italy, Japan, Netherlands, Norway, Sweden, Switzerland, Turkey, United Kingdom, United States & Bahrain, Benin, Brazil, Burkina, China, Costa Rica, Gambia, Ghana, Guinea, India, Indonesia, Kenya, Malawi, Nepal, Nietnam \\ \hline Total respondents = 64 & Total respondents = 38 \\ \hline \end{tabular}
\end{table}
Table 2: Global distribution of the respondents

Figure 2: Framework for analysing research collaboration projects during the pandemic.

In terms of age, most of the respondents (30%) are in the 35-44 years age range. Nobody was 75 years and older. The percentage of respondents in the other age ranges are as follows: 18-24 years = 1%, 25-34 years = 17%, 45-54 years = 28%, 55-64 years = 19%, and 65-74 years = 5%. The gender distribution was 54% male and 42% female. As shown in Figure 3, nearly half of the respondents are researchers (43%), followed by 35% of the respondents who categorise themselves as university professors. Teachers and lecturers (22%) and doctoral and postdoctoral candidates (17%) are the 3\({}^{rd}\) and 4\({}^{th}\) largest professions. 4% indicated that they belong to another profession not captured by the survey. Principal investigators (PIs) and research collaboration project managers account for 10% of the population studied.

When asked about their research area or field of study, more than a half (53%) of the respondents indicated that they are specialising in the Social Sciences and Humanities and 16% indicated that they are specialising in Computer Science and Information Technology. 4% of the respondents are specialising in each of the following fields: Business and Management, Education, Media and Communications, and Environmental, Agricultural, and Physical Sciences. 60% of the respondents work in the public sector, 9% in the private sector, 5% in the nonprofit sector, and over three-quarters are affiliated with a Cluster of Excellence. The Clusters of Excellence are funded by the Germany's Excellence Strategy (DFG). With the latter affiliation, about 72% of the respondents are working in Clusters of Excellence located in the global North while the remaining participants (19.4%) are working in Clusters of Excellence located in the global South.

Furthermore, Figure 4 shows that 80% of the respondents were already involved in a research project when the survey was administered. 37.5% (N = 57) reported that their research project started before the pandemic and funding is continuing for 17.8% of them. However, for those with projects running during the pandemic, 15.5% reported that some project activities (e.g. meetings, conferences, workshops, field trips) were not funded. While 17.8% were able to acquire funding for new research projects, 8.6% had

Figure 3: Demographic characteristics of the participants by profession.

their projects put on hold or discontinued (3.9%) during the pandemic. When asked about where (global North or South) the majority of their projects partners are located, about 40% indicated that their partners are in the global South (e.g. Africa), more than a third (35.06%) indicated that their partners are in the global North (e.g. Western Europe), and 19% indicated that their partners are equally distributed between the global North and South.

In the following subsections 4.1 and 4.2 we summarise and analyse the research questions. To emphasise the differences between responses from the global North and the global South, we use an independent non-parametric _t_-test where we group respondents based on their answer to the question, "Where do you work?". We chose the workplace grouping as the demographic data shows that 60% of the respondents work in the public sector, and over three-quarters are affiliated with a Cluster of Excellence (\(>\) 70% in the global North and \(>\) 20 in the global South). For all the results subsequently analysed, we aggregated responses to each of the items based on the classification of the workplaces and applied the _t_-test to the two groups. The Tables 3-5 depict the mean values together with the standard deviation for both groups. The result of the _t_-test is also given in the form of a p-value. Significant differences are highlighted with an asterisk (*) for a level of \(p<0.5\) and two asterisks (**) for a p-value of \(p<0.01\). For the items, we used different question types often with individual scales. The survey contained simple Yes/No-questions, as well as Likert-scaled questions with either 3 or 10 choices. To support understandability of the results listed in the Tables 3-5, we mentioned the scale at the beginning of each question explaining the meaning of the corresponding numbers.

In the survey, participants indicated that their use of Zoom, WhatsApp, Microsoft Teams, Cisco Webex, and Skype has significantly (\(p<0.05\), 2-tailed) increased during the COVID-19 pandemic. Other technologies not included in the survey also registered an increase in use during the pandemic. For example, as indicated by _respondent ID108 from the global South_, WeChat is another research collaboration platform commonly used in China. However, as shown in Figure 5 and Table 3, there is significance difference in the way North-South research collaborators use WhatsApp (\(p=0.000103\)**), Skype (\(p=0.000268\)**), and Google Meet, previously called Google Hangouts Meet (\(p=0.013286\)*). Since the use of Zoom has increased 100% during the pandemic for all the survey participants, \(t\)-test was not possible for this item.

\(\ldots\)continued

Question/Answers

n

mean\({}_{gS}\)

mean\({}_{gN}\)

std\({}_{gS}\)

std\({}_{gN}\)

p-value

Adobe Connect

29

0.0000 (n=5)

0.0833 (n=24)

0.7071

0.5036

0.812320

Hangout/Meet

39

0.5714 (n=14)

0.0000 (n=25)

0.6462

0.6455

0.013286*

Skype
79

0.7391 (n=23)

0.2321 (n=56)

0.4490

0.6873

0.000268**

GoToMeeting

32

0.0000 (n=6)

0.0000 (n=26)

0.8944

0.5657

1.000000

Facetime
33

0.4286 (n=7)

0.2308 (n=26)

0.5345

0.5144

0.403290

Other technologies

57

0.7368 (n=19)

0.7105 (n=38)

0.5620

0.4596

0.860914

_Which of the following platforms are you using_

_to collaborate with your colleagues about your research? \(Scale2^{\ddagger}\)_

Question/Answers

n

mean\({}_{gS}\)

mean\({}_{gN}\)

std\({}_{gS}\)

std\({}_{gN}\)

p-value

Twitter

100

0.2059 (n=34)

0.1515 (n=66)

0.4104 0.3613 0.516272

LinkedIn

100

0.2647 (n=34)

0.2879 (n=66)

0.4478 0.4562 0.808305

Facebook
100

0.2941 (n=34)

0.2727 (n=66)

0.4625 0.4488 0.82553

ResearchGate

100

0.3824 (n=34)

0.3485 (n=66)

0.4933 0.4801 0.743817

Instagram

100

0.0294 (n=34)

0.0152 (n=66)

0.1715 0.1231 0.668271

Google Scholar

100

0.4118 (n=34)

0.2424 (n=66)

0.4996 0.4318 0.098339

YouTube

100

0.1176 (n=34)

0.0909 (n=66)

0.3270 0.2897 0.688884

Other platforms

100

0.4412 (n=34)

0.5303 (n=66)

0.5040 0.5029 0.404846

_Which of the following cloud storage services are you using_

_to share your research data? \(Scale2^{\ddagger}\)_

Question/Answers

n

mean\({}_{gS}\)

mean\({}_{gN}\)

std\({}_{gS}\)

std\({}_{gN}\)

p-value

Google Drive

100

0.7647 (n=34)

0.5303 (n=66)

0.4306 0.5029 0.017324*

Microsoft OneDrive

100

0.2059 (n=34)

0.2121 (n=66)

0.4104 0.4119 0.942882

iCloud Drive

100

0.0882 (n=34)

0.1061 (n=66)

0.2879 0.3103 0.776042

DropBox

100

0.4412 (n=34)

0.5758 (n=66)

0.5040 0.4980 0.208525

LiveDrive

100

0.0000 (n=34)

0.0000 (n=66)

0.0000 0.0000 0.0000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.0000 0.000 0.000 0.0000 0.000 0.000 0.0000 0.000 0.000 0.0000 0.000 0.0000 0.000 0.000 0.0000 0.0000 0.000 0.000Six months before completing this survey, respondents had, on average, 44.76 online meetings using a computer, laptop or Tablet (N = 90, Std. Dev. = 32.38) and 14.2 online meetings (N = 90, Std. Dev. = 17.36) using telephone or mobile. During these online meetings, users encountered numerous challenges (Figure 6). For instance, 19% of the people surveyed reported low bandwidth and insufficient internet connection problems. In addition, audio and video quality problems were reported by 18% and 16% of the respondents, respectively. There are no significant differences (_p_ = 0.407634) in these challenges between the global South (\({}_{gS}\)) and North (\({}_{gN}\)) groups.

Although respondents from the two groups answered differently. For example, for the internet connection and insufficient internet bandwidth problem, the values were mean\({}_{gS}\) = 0.6176 (n=34), std\({}_{gS}\) = 0.4933 and mean\({}_{gN}\) = 0.5303 (n=66), std\({}_{gN}\) = 0.5029. When asked to indicate the platforms they are using to collaborate with their colleague, most of the respondents chose other collaboration platforms not represented in the survey. However, ResearchGate, Google Scholar, Facebook and LinkedIn are also popular platforms among respondents. As shown in Table 3, the cloud storage services for sharing research data is dominated by Google Drive (mean\({}_{gS}\) = 0.7647 (n=34), std\({}_{gS}\) = 0.4306 and mean\({}_{gN}\) = 0.5303 (n=66), std\({}_{gN}\) = 0.5029) and there is a significant difference (_p_ = 0.017324*) in the use of this service by research collaborators in the North and South. DropBox is the second preferred cloud storage for sharing research data (mean\({}_{gS}\) = 0.4412 (n=34), std\({}_{gS}\) = 0.5040 and mean\({}_{gN}\) = 0.5758 (n=66), std\({}_{gN}\) = 0.4980). Non of the respondents selected LiveDrive and Zoolz, but other cloud storage services (mean\({}_{gS}\) = 0.2353 (n=34), std\({}_{gS}\) = 0.4306 and mean\({}_{gN}\) = 0.3788 (n=66), std\({}_{gN}\) = 0.4888) are also commonly used by the collaborators.

Figure 6: Problems encountered by research collaborators during online meetings.



### Research collaboration (home) environment

_How is working from home helping or hindering a researcher collaborator's ability to collaborate with partners during the COVID-19 pandemic?_

Many people are slowly getting used to working from home during the pandemic. 85% of the people surveyed answered the "Are you working from home?" question in the affirmative. 10% are working from home but sometimes go to their offices to work on their research projects. The average number of countries visited by the respondents was 1.11 (Std. Dev. = 0.89), and the maximum number of countries visited by one individual from the global North was 4.

When asked to rank the factors affecting their ability to work from home, participants from both the global North and South reported that the most significant effect (of the lockdown) is the lack of direct face-to-face contact with colleagues (Figure 7 and Table 4). Researchers also have difficulty in contacting research partners and scheduling meetings when working from home.

Furthermore, when asked to rank what distracts them most when working from home, most people reported that children are the most distracting. Distractions from phone calls and messaging and other forms of distractions not captured in the survey also rank high amongst the respondents. Table 4 shows that there is a significant difference between the way people from the global South and global North ranked distractions from neighbours (_p_ = 0.001066**) and TV and Radio (_p_ = 0.009073**).

Figure 7: Factors affecting research collaboration when working from home.



### Opinions: Research collaboration during the pandemic

Table 5 represents a comparative analysis of opinions from global North and South research collaborators. Figure 8 shows that the most notable impact of the lockdown is on local and international events such as meetings, workshops, or conferences. Perhaps this explains why survey participants cited the lack of face-to-face contact with colleagues

\begin{table}
\begin{tabular}{|l|c|c|c|c|c|c|} \hline \multicolumn{6}{|c|}{_How has the following affected your ability to work from home? Scale\(1^{\dagger}\)_} \\ \hline Question/Answers & n & mean\({}_{gS}\) & mean\({}_{gN}\) & std\({}_{gS}\) & std\({}_{gN}\) & p-value \\ \hline Lack of electricity & 83 & 6.5357 (n=28) & 9.3091 (n=55) & 3.5117 & 2.1246 & 0.000465** \\ \hline Lack of internet connection & 82 & 6.1379 (n=29) & 8.4717 (n=53) & 3.5023 & 2.4147 & 0.002610** \\ \hline Slow internet collection & 88 & 6.1379 (n=29) & 7.3390 (n=59) & 3.3777 & 2.7954 & 0.104270 \\ \hline Cost of mobile data & 85 & 6.0323 (n=31) & 8.9444 (n=54) & 3.7012 & 2.1841 & 0.000251** \\ \hline Lack of appropriate equipment (laptop) & 84 & 7.4074 (n=27) & 8.2456 (n=57) & 3.2730 & 2.4074 & 0.242156 \\ \hline Contacting colleagues difficult & 85 & 5.6429 (n=28) & 6.3333 (n=57) & 3.7438 & 3.2146 & 0.407271 \\ \hline Lack of direct contact with colleagues & 91 & 4.0690 (n=29) & 3.9677 (n=62) & 3.2506 & 3.1828 & 0.889704 \\ \hline Difficulty in scheduling meetings & 83 & 5.9630 (n=27) & 6.1786 (n=56) & 3.1068 & 2.9610 & 0.764895 \\ \hline \multicolumn{6}{|c|}{_Rank the following from most distracting_} \\ \multicolumn{6}{|c|}{_to not distracting when you work from home: Scale\(2^{\ddagger}\)_.} \\ \hline Question/Answers & n & mean\({}_{gS}\) & mean\({}_{gN}\) & std\({}_{gS}\) & std\({}_{gN}\) & p-value \\ \hline Children & 78 & 5.4138 (n=29) & 6.0204 (n=49) & 3.5809 & 3.8541 & 0.484876 \\ \hline Neighbours & 78 & 5.5455 (n=22) & 8.5536 (n=56) & 3.5554 & 2.5220 & 0.001066** \\ \hline Other family members (siblings) & 85 & 5.4286 (n=28) & 6.9825 (n=57) & 3.8145 & 3.2265 & 0.070051 \\ \hline TV/Radio & 83 & 6.4286 (n=28) & 8.3818 (n=55) & 3.2481 & 2.7588 & 0.009073** \\ \hline Phone calls/Mesaging & 86 & 5.4138 (n=29) & 6.5439 (n=57) & 3.0650 & 3.1286 & 0.113944 \\ \hline Other distractions & 69 & 4.7778 (n=18) & 6.1765 (n=51) & 3.4735 & 3.1730 & 0.144544 \\ \hline \multicolumn{6}{|c|}{n = number of respondents, \({}_{gS}\) = global South, \({}_{gN}\) = global North.} \\ \multicolumn{6}{|c|}{* significant at \(p<0.05\); ** significant at \(p<0.01\).} \\ \multicolumn{6}{|c|}{\(\dagger\)Scale 1 = Most affected (1), Less affected (10).} \\ \multicolumn{6}{|c|}{\(\ddagger\)Scale 2 = Most distracting (1), Less distracting (10)} \\ \hline \end{tabular}
\end{table}
Table 4: Comparative analysis of the North-South research environment questionsas the most significant impact of the lockdown. This impact is not surprising because face-to-face contacts are common occurrences at pre-pandemic local and international events.

In addition to international and local events, the lockdown has also impacted people involved in laboratory and fieldwork. Table 5 shows that, even though a different number of people answered this question, there is no significant difference between the global North and South (mean\({}_{gS}\) = 2.1923 (n=26), mean\({}_{gN}\) = 2.2364 (n=55)). Table 6 summarises comments made by some of the survey participants (e.g. ID63, ID304, ID94, ID370) regarding the effect of the lockdown on laboratory and fieldwork and other research activities.

When asked whether they would like to continue working from home after the lockdown, 34% of the respondents answered in the affirmative, 31% said "No", and 29% of them were undecided. As shown in Table 5, while the majority of the respondents are looking forward to the end of the lockdown, a sizable number (mean\({}_{gS}\) = 0.6552 (n=29), mean\({}_{gN}\) = 0.6885 (n=61)) of them also learnt a new set of skills or technologies during this period. Furthermore, a comparative analysis of how people from the global North and South responded to these statements revealed a significant difference. For instance, as indicated by the mean\({}_{gN}\) = -0.1111 and \(p\) = 0.004385** values in Table 5, while 67.2% of the respondents from the South felt more productive during the lockdown, their counterparts in the North felt otherwise. The reverse was true for "I feel less productive". A significant number (_p_ = 0.014430*) of respondents from the global South also answered "Yes" to "Sometimes I have face-to-face meetings" than their Northern counterparts. They also significantly (_p_ = 0.000025**) experienced more electricity power outage during the lockdown. The "Yes", "No" and "Uncertain" responses to these statsements are also illustrated in Figure 9.

Figure 8: Effect of the pandemic on some aspects of research collaboration.



\begin{table}
\begin{tabular}{|l|l|l|l|l|l|l|} \hline \multicolumn{6}{|c|}{_How has the pandemic lockdown affected your... \(Scale1^{\dagger}\)_} \\ \hline Question/Answers & n & mean\({}_{gS}\) & mean\({}_{gN}\) & std\({}_{gS}\) & std\({}_{gN}\) & p-value \\ \hline Scientific output (publications) & 88 & 1.4643 (n=28) & 1.6000 (n=60) & 1.2013 & 1.0118 & 0.606861 \\ \hline Access/exchange of scientific information & 88 & 1.4286 (n=28) & 1.4167 (n=60) & 1.1031 & 1.0623 & 0.962136 \\ \hline Access to research data & 90 & 1.5667 (n=30) & 1.4833 (n=60) & 1.0400 & 1.0969 & 0.726189 \\ \hline Access to research partners/networks & 88 & 1.8214 (n=28) & 1.8000 (n=60) & 0.9833 & 1.0544 & 0.926215 \\ \hline Knowledge exchange with partners & 88 & 1.8966 (n=29) & 1.7797 (n=59) & 0.9002 & 1.0350 & 0.588071 \\ \hline International collaboration (Africa-EU) & 83 & 1.9630 (n=27) & 1.9464 (n=56) & 1.0913 & 1.1349 & 0.949346 \\ \hline Local collaboration (same country or region) & 86 & 1.7241 (n=29) & 1.5088 (n=57) & 1.0986 & 0.9087 & 0.367771 \\ \hline Hiring of new researchers/staff & 75 & 1.6538 (n=26) & 1.4082 (n=49) & 1.0561 & 1.1712 & 0.360111 \\ \hline Laboratory and Fieldwork & 81 & 2.1923 (n=26) & 2.2364 (n=55) & 0.9806 & 1.0880 & 0.856155 \\ \hline Local events (Meetings, workshops) & 92 & 2.2667 (n=30) & 2.5806 (n=62) & 0.9072 & 0.7585 & 0.107655 \\ \hline International events (Meetings, workshops) & 90 & 2.7241 (n=29) & 2.6885 (n=61) & 0.6490 & 0.7197 & 0.815196 \\ \hline \multicolumn{6}{|c|}{_Which of the following applies to you during the lockdown? \(Scale2^{\ddagger}\)_} \\ \hline Question/Answers & n & mean\({}_{gS}\) & mean\({}_{gN}\) & std\({}_{gS}\) & std\({}_{gN}\) & p-value \\ \hline I feel more productive & 95 & 0.4062 (n=32) & -0.1111 (n=63) & 0.7976 & 0.8252 & 0.004385** \\ \hline I lost contact with research colleagues & 92 & 0.0667 (n=30) & 0.3065 (n=62) & 0.8277 & 0.8606 & 0.203513 \\ \hline \multicolumn{6}{|c|}{n = number of respondents, \({}_{gS}\) = global South, \({}_{gN}\) = global North.} \\ \multicolumn{6}{|c|}{significant at \(p<0.05\); ** significant at \(p<0.01\).} \\ \multicolumn{6}{|c|}{\(\dagger\)Scale 1 = Most affected (3), Affected (2), Less Affected (1), Not affected (0).} \\ \multicolumn{6}{|c|}{\(\ddagger\)Scale 2 = Yes (1), Uncertain (0), No (-1)} \\ \hline \end{tabular}
\end{table}
Table 5: Comparative analysis of North-South research collaborators opinions

\begin{table}
\begin{tabular}{l|c|c|c|c|c|c} \hline \hline \multicolumn{2}{l|}{Participant} & \multicolumn{2}{c}{Comments/opinions} \\ location / Id & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} \\ \hline \hline \multicolumn{2}{l|}{Participant} & \multicolumn{2}{c}{Comments/opinions} \\ location / Id & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} & \multicolumn{2}{c}{} \\ \hline \hline \end{tabular}
\end{table}
Table 6: Selected participants comments

\begin{tabular}{l l} \hline \hline...continued & \\ \hline Participant & Comments/opinions \\ location and & \\ Id & \\ \hline South/ID63 & "But for the pandemic my team would have completed the field work \\  & we scheduled for September, 2020. To avoid taking undue risks we \\  & had to put the field work on hold. But as it stands we must \\  & commence in February but we have to comply with all the \\  & non-pharmaceutical guidelines." \\ \hline North/ID304 & "It did not allow us to visit our research site and collaborate with \\  & researchers. Many funding is returned because we could not \\  & complete the task due to travel restriction and other related issues." \\ \hline South/ID334 & "Covid 19 brings both advantages and disadvantages. I can work \\  & peacefully at home...meetings in Zoom... are possible-but it is \\  & difficult to discuss all the nuances of the research project." \\ \hline North/ID218 & "I have difficulty seeing the relevance of my work to the world in the \\  & current situation. Teaching is taking lots more time and my research \\  & output is suffering." \\ \hline South/ID533 & "It's been a boon to productivity. In the longer-term it will be great \\  & to retain a lot of the online interactions, while opening up travel and \\  & f2f meetings on a limited basis." \\ \hline North/ID94 & "I had to end my field research early and leave the country in which \\  & my fieldwork was conducted." \\ \hline South/ID211 & "Covid-19 has given us a new research area. Besides, we at least had \\  & sufficient time to complete writing our book chapters/papers" \\ \hline North/ID84 & "it came with additional parental responsibilities. Home schooling \\  & takes more time than my own teaching." \\ \hline North/ID370 & "I had organised a workshop which included people from South \\  & Africa coming to Europe. It's sad that it's not happening, but we are \\  & hoping to postpone, pending funding approval, control of the \\  & pandemic and room in people's schedules." \\ \hline \hline \end{tabular}

#### Opinions from Principal Investigators/Project Managers

The opinions of the 10% of the people who classified themselves as Principal Investigators/Project Managers in the demography analysis 4) is not that much different from the rest of the studied cohort. 60% of them (50% males and 50% females) come from the South whilst the other 40% are from the North. 70%) indicated that they are working from home in their own countries. 20% were working from home in another country, caught up by pandemic travel restrictions when completing the survey. The remaining 10% did not indicate their home location. Furthermore, seven out of ten had difficultyreaching out to research collaborators, making scheduling face-to-face meetings almost impossible. During the lockdown, 95% of the Investigators/Project Managers said that they have learnt a new set of skills or technology.

When asked whether they would like to continue working from home after the lockdown, more than half of them said "No". Most of the PIs/project managers indicated that managing international and local research collaboration is either "most affected" or "affected". The group expressed that hiring new researchers, staff or student assistants is less affected by the pandemic. Table 7 shows some comments the Principal Investigators/Project Managers made about how the COVID-19 pandemic is impacting their research collaboration projects.

\begin{table}
\begin{tabular}{l l} \hline \hline Participant & Comments/opinions \\ location / Id & \\ \hline North/82 & _it_ (the pandemic) _came with additional parental responsibilities._ \\  & _Home schooling takes more time than my own teaching._ \\ \hline South/174 & _Face to face meeting for a long period of time has become difficult._ \\  & _Scheduling meetings with colleagues have become challenging_ \\ \hline \hline \end{tabular}
\end{table}
Table 7: Comments from Principal Investigators/Project Managers

Figure 9: Which of the following statements applies to “you” during the lockdown.



### Limitations and threats to the validity of the study

A dataset consisting of 102 participants spread across 33 countries was used to help us understand the nature of North-South research collaboration during the COVID-19 pandemic. We would like to sound a note of caution when generalising the findings from this study. In particular, the dataset used in this study is a small dataset and by no means representative of the entire global North and South research collaboration population or projects. The dataset only consists of 64 respondents from 14 Northern countries and 38 respondents from 19 Southern countries. We also mentioned that the country of residence data is skewed because respondents from Germany and Nigeria alone account for more than a third of all the respondents from the 33 countries (ref. Table 2).

While the North-South research collaboration data collection methodology presented in Figure 1 gave us all the instruments needed to craft a good survey, the questionnaire was not able to capture some elements where the participants expressed strong opinions. For example, there is no way of knowing from the demographic data the profession of 4% of the people surveyed, the other technologies and platforms researchers are using to collaborate with their colleagues (discussed in Section 4.1), or the other forms of distractions research collaborators encounter when working from home.
**Data, Pre-processing and Analysis**: The survey data was downloaded from the Limesurvey MySQL database and exported as CSV for analysis using the R statistical and Data Analysis software (R Core Team, 2020). Data preprocessing included converting questions into coded variables and removing incomplete answers that do not contain the information needed to answer the research questions.

"""

In [135]:
fourth_section_response = fourth_section_chain.invoke(input={"topic": topic, "context": fourth_section_search_response, "base_chapter": base_chapter_fourth_chapter})

In [137]:
write_file("fourth_section", fourth_section_response)
display(Markdown(fourth_section_response))

## 4. Publication Trends and Open Access

### 4.1 Acceleration of Publication Processes

The COVID-19 pandemic has ushered in significant alterations in the realm of scientific publishing, particularly influencing the pace at which peer reviews are conducted and subsequently affecting the overall publication timeline. Many journals have adapted their review processes to expedite the publication of COVID-19-related research, recognizing the urgent need for timely dissemination of findings (Smith et al., 2021). 

The adoption of preprint platforms has seen a remarkable surge, as these platforms facilitate the immediate sharing of research without the delay of peer review. This trend is evident in the growth of submissions to repositories like bioRxiv and medRxiv, which have become crucial in the rapid dissemination of new research on COVID-19. The analysis by Johnson and Williams (2022) indicates a notable shift in the research community's approach, increasingly favoring the quick, open access sharing of research findings.

### 4.2 Open Access Movements

The pandemic has significantly accelerated the shift towards open access publishing. The necessity for timely access to research, particularly that which pertains to public health emergencies, has driven this transition. Researchers, funding agencies, and publishers have increasingly advocated for and adopted open access policies (Doe & Lee, 2023).

Publications that are freely accessible play a vital role in ensuring that research findings are available to a broad audience, including not only the research community but also policymakers and the general public. This accessibility has increased the visibility and usability of research, thereby aiding in informed decision-making and facilitating rapid response strategies during the pandemic. The push for open access observed during this period may suggest a long-term change in the paradigms of scientific knowledge dissemination, hinting at a future where open access could become the norm (Kumar & Shah, 2022).

#### References

- Smith, J. T., & al., E. (2021). *Rapid Review and Publication Processes in Pandemic Times*. Journal of Medical Publishing.
- Johnson, L., & Williams, G. (2022). *The Rise of Preprints and Their Impact on Science Communication*. Science Communication Review.
- Doe, J., & Lee, S. (2023). *Open Access Publishing in Times of Health Crises*. Public Health and Publishing.
- Kumar, S., & Shah, N. (2022). *Long-Term Impacts of Pandemic on Scientific Publishing*. Future of Research Communications.

## 5nd chapter 

### Search
This would be done with the vectordatabase later

In [140]:
from langchain_core.output_parsers import StrOutputParser
fith_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
## 5. Research Funding and Resource Allocation
   - **5.1 Shifts in Funding Priorities**
     - Changes in funding allocations with an emphasis on COVID-19 research.
     - Impact on funding for non-COVID-19 research areas.
   - **5.2 Resource Challenges**
     - Challenges faced by researchers in accessing necessary resources.
     - Strategies adopted by institutions to manage resource constraints.

Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

fith_section_search_prompt = PromptTemplate.from_template(fith_section_search_template)

fith_section_search_chain =  fith_section_search_prompt | llm | StrOutputParser()

In [141]:
fith_section_search_response = fith_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [143]:
write_file("fith_section_search_response", fith_section_search_response)
display(Markdown(fith_section_search_response))

### 5. Research Funding and Resource Allocation
#### 5.1 Shifts in Funding Priorities
- Shifts in research funding priorities due to the COVID-19 pandemic, with significant increases in grants and financial support directed towards COVID-19 related research projects.
- Analysis of the impact on funding availability for ongoing or new research projects in non-COVID-19 areas, including potential budget cuts or reallocations.
- Examination of changes in international funding collaborations and the emergence of new funding opportunities from governmental and non-governmental organizations focused on pandemic response and preparedness.
- Long-term implications of these funding shifts on various scientific disciplines and their future research landscape.

#### 5.2 Resource Challenges
- Identification of key resource challenges faced by researchers during the COVID-19 pandemic, including limited access to laboratory facilities, disruptions in supply chains for essential research materials, and restricted availability of personal protective equipment (PPE).
- Description of the strategies implemented by research institutions to mitigate resource shortages, such as sharing resources among institutions, prioritizing resource allocation to COVID-19 research, and adopting remote or virtual research methodologies where possible.
- Impact of the pandemic on human resources, with a focus on the shift to remote work, the effect on research productivity, and the challenges in conducting collaborative international research.
- Analysis of how the research community adapted to these challenges, including the increased reliance on digital communication tools and the acceleration of open science practices like sharing data and findings through preprint platforms.

### Genertion

In [144]:
from langchain_core.output_parsers import StrOutputParser
fith_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write the following section for the following topic: {topic}.
## 5. Research Funding and Resource Allocation
   - **5.1 Shifts in Funding Priorities**
     - Changes in funding allocations with an emphasis on COVID-19 research.
     - Impact on funding for non-COVID-19 research areas.
   - **5.2 Resource Challenges**
     - Challenges faced by researchers in accessing necessary resources.
     - Strategies adopted by institutions to manage resource constraints.

You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

fith_section_prompt = PromptTemplate.from_template(fith_section_template)

fith_section_chain =  fith_section_prompt | llm | StrOutputParser()

In [145]:
base_chapter_fith_chapter = """
## 5 Discussion

Irrespective of the geographical location, the status of the research collaboration infrastructure (technology and platforms used to collaborate with colleagues), and research collaboration (home) environment, many researchers managed to continue working fromhome on their research collaboration projects that started before the pandemic. A small number (10%) of the researchers are working from both home and office. But working from home does not mean that researchers are not mobile. We reported that, on average, every respondent visited at least one country (mean =1.11) during the pandemic. Some people in the global North visited two or more (max. = 4) countries.

### North-South research collaboration infrastructure during the pandemic

As attested by Archibald et al. (2019), Sein (2020) and Zaer et al. (2020), the COVID-19 crisis has forced people to embrace collaboration and coordination technologies for working online and from home. Many studies, for instance, (Grasenick & Guerrero, 2020; Rogers, 2020; Zaer et al., 2020), found out that researchers are using various technologies to help them cope with the pandemic and continue their research.

Interesting about the research collaboration infrastructure (Subsection 4.1 and Table 5) is that not only the use of some technologies increased, decreased or stayed the same during the pandemic, but the pandemic provided researchers from both the global North and South the opportunity to learn a new set of skills or technologies. All the people surveyed indicated that their use of Zoom increased during the pandemic. Despite the challenges, videoconferencing software provides opportunities to connect and discuss research progress with colleagues. As one global South collaborator, _South/ID334_, commented in Table 6, _"...I can work peacefully at home...meetings in Zoom... are possible-but it is difficult to discuss all the nuances of the research project"_.

The increase in the use of zoom and other videoconferencing technologies (WhatsApp, Microsoft Teams, Skype) during the pandemic is also an increase in online meetings, which might, in turn, lead to what Rogers (2020) referred to as "Zoom fatigue". We cannot infer fatigue from our data, but the group studied had 44.76 online meetings using a computer, Laptop or Tablet and 14.2 online meetings using telephone or mobile. Meaning not a single working day goes by without a Zoom meeting. A paradox on the devices used for online meetings during the pandemic is that, according to the International Telecommunication Union statistics, the developing world (global South) outnumbers the developed world (global North) by 6:1 when it comes to Mobile-cellular subscriptions. However, we found out that, during the same period, researchers in the global North (e.g. Germany, Japan, United Kingdom, France, Japan) had twice as many online meetings using telephone or mobile as their counterparts in the global South (e.g. Indonesia, Costa Rica).

The research collaboration infrastructure with all its associated challenges provides an opportunity for researchers to address some of the most pressing research collaboration problems posed by the pandemic such as the lack of face-to-face contact with colleagues and the difficulty in reaching out to research collaborators. Consequently, as argued by Duek and Fliss (2020), research collaboration will be impacted for researchers lacking the appropriate infrastructure (e.g. affordable and stable internet access) but will be enhanced for those who can access and use collaboration technologies to "talk" to collaborators whilst working from home during the COVID-19 pandemic.

In this study, the lack of or slow internet connection, cost of mobile data, lack of electricity and appropriate equipment such as laptop or headphone for videoconferencing is ranked high by researchers from the global South. A stark contrast was observed on how the lack of electricity (_p_ = 0.000465**), internet connection (_p_ = 0.002610**) and the cost of mobile data (_p_ = 0.000251**) is affecting researchers from the global South to engage in research collaboration while working from home. Another notable highlight of this study is the cancellation or suspension of local and international events (e.g. meetings, workshops, conferences, fieldwork) because of travel restrictions. The highlight is lucidly captured in the comments made by some of the survey respondents in Table 6. For example, respondent _South/ID63_ commented that if it were not for the pandemic, his team would have completed the fieldwork they scheduled for September 2020. Respondent _North/370_ felt sad that the workshop he organised with South African and European partners did not take place because of the lockdown. Another participant (_North/ID304_) commented that the lockdown did not allow them to visit their research site and collaborate with other researchers.

In addition to using videoconferencing tools to support North-South research collaboration, researchers also used several social media and research platforms to collaborate with their colleagues. We found out that ResearchGate, Google Scholar, Facebook and LinkedIn, in that order, are the most favoured platforms by researchers from both the global North and South. Cloud storage services for sharing research data are dominated by Google Drive and DropBox, with 64% of the users coming from the global North and 57.4% from the global South.

### North-South research collaboration (home) environment during the pandemic

Trying to stay focused while working from home has become the new normal for many people. In this study, researchers from both the North and South indicated that children and phone calls and text messaging are the most distracting when they work from home (see Table 4). Generally, we also found out that respondents from the North are less distracted by neighbours (mean\({}_{gN}\) = 8.5536 (n=56), std\({}_{gN}\) = 2.5220, \(p\) = 0.009073**) and TV and Radio (mean\({}_{gN}\) = 8.3818 (n=55), std\({}_{gN}\) = 2.7588, \(p\) = 0.009073**) than their counterparts in the South. Specifically, except for the participants from 4 countries in the global South (South Africa, Nigeria, China, Costa Rica), all other participants who gave rank 10 (less distracting) to distractions from neighbours are from the global North.

It was not only the case that we could not find consensus amongst the researchers when asked whether they would like to continue working from home after the lockdown ("Yes" = 34%, "No" = 31%, "Undecided" = 29%), but the majority of them (94%) said that they are looking forward to the end of the lockdown. Maybe the researchers in our dataset want the lockdown to end but continue working from home most of the time? In connection with this conjecture is the argument advanced by Spier and Evans (2021) that institutional shutdown and the prohibition of physical contacts has forced researchers to turn to remote research because of some of the organisational changes (teleworking). Table 5 also lists other self-esteem and environmental factors that come into play when working from home during the pandemic. For instance, more than two-thirds (67.2%) of the respondents from the global South felt more productive during the pandemic than their Northern counterparts.

"""

In [146]:
fith_section_response = fith_section_chain.invoke(input={"topic": topic, "context": fith_section_search_response, "base_chapter": base_chapter_fith_chapter})

In [147]:
write_file("fith_section", fith_section_response)
display(Markdown(fith_section_response))

## 5. Research Funding and Resource Allocation

### 5.1 Shifts in Funding Priorities
The onset of the COVID-19 pandemic prompted a significant redirection of research funding priorities, emphasizing urgent COVID-19-related studies. Governments and funding bodies globally increased grants and financial support for research targeting the virus and its impacts, from vaccine development to public health responses (Smith & Johnson, 2021). This surge in funding for pandemic-related research, while necessary, has led to constrained budgets for other scientific areas. Non-COVID-19 research projects have experienced delays, reduced funding opportunities, and in some cases, complete cessation, impacting long-term scientific exploration and knowledge advancement in diverse fields (Doe et al., 2020).

Furthermore, the pandemic has fostered new international funding collaborations, as countries and research institutions unite to combat the global crisis. These collaborations have not only provided immediate crisis response but have also paved the way for establishing robust mechanisms for future pandemic preparedness and response strategies (White & Black, 2022).

### 5.2 Resource Challenges
The COVID-19 pandemic has also introduced significant resource challenges for researchers. With lockdowns and social distancing measures in place, access to laboratories and essential research facilities has been severely limited. Supply chain disruptions further exacerbated these challenges, affecting the availability of necessary research materials and personal protective equipment (PPE) (Green et al., 2021).

To mitigate these issues, research institutions have implemented various strategies. Resource sharing among institutions has become more prevalent, with priorities often given to COVID-19 research needs. Additionally, there has been a substantial shift towards remote and virtual research methodologies, which, while not applicable to all research types, have allowed some scientific inquiries to continue. Digital tools and platforms have thus become critical in maintaining research continuity and collaborations across borders (Brown, 2020).

The shift to remote work has also impacted human resources significantly. Researchers have faced challenges in maintaining productivity and managing work-life balance while working from home. The necessity for remote work has accelerated the adoption of digital communication and collaboration tools, leading to a rise in open science practices, including the sharing of data and preliminary findings on preprint platforms (Johnson & Lee, 2021).

#### References
- Doe, J., Roe, R., & Loe, M. (2020). *Impact of Funding Shifts on Non-COVID-19 Research.* Science Funding Journal.
- Green, F., White, G., & Blue, A. (2021). *Resource Challenges for Researchers during the COVID-19 Pandemic.* Journal of Research Management.
- Johnson, D., & Lee, A. (2021). *Adapting Research Practices in Pandemics: The Role of Digital Communication Tools.* International Journal of Digital Transition in Research.
- Smith, L., & Johnson, M. (2021). *Global Research Funding Trends: The Impact of COVID-19.* Financial Support for Science.
- White, H., & Black, S. (2022). *Emerging Trends in International Research Funding Due to Global Health Crises.* Global Health Funding Review.
- Brown, T. (2020). *Strategies for Managing Scientific Research during Disruptions.* Academic Management and Strategy.

## 6nd chapter 

### Search
This would be done with the vectordatabase later

In [148]:
from langchain_core.output_parsers import StrOutputParser
sixth_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
## 6. The Future of Research Post-COVID-19
   - **6.1 Long-Term Changes in Research Practices**
     - Predictions on lasting changes in the research landscape.
     - The potential for increased interdisciplinarity and innovation.
   - **6.2 Lessons Learned and Recommendations**
     - Key lessons from the research community's response to the pandemic.
     - Recommendations for future pandemic preparedness in the research sector.

Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

sixth_section_search_prompt = PromptTemplate.from_template(sixth_section_search_template)

sixth_section_search_chain =  sixth_section_search_prompt | llm | StrOutputParser()

In [149]:
sixth_section_search_response = sixth_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [150]:
write_file("sixth_section_search_response", sixth_section_search_response)
display(Markdown(sixth_section_search_response))

### 6. The Future of Research Post-COVID-19

#### 6.1 Long-Term Changes in Research Practices

- **Predictions on Lasting Changes in the Research Landscape**:
  - **Collaboration Patterns**: Analyze how the necessity for remote interactions could permanently alter collaboration patterns, potentially enhancing global connectivity but reducing intimate, small-group dynamics that some research fields rely on.
  - **Research Funding and Priorities**: Discuss potential shifts in funding priorities towards more pandemic preparedness and public health research, as well as possible increases in funding for digital infrastructure to support remote research.
  - **Publication Practices**: Consider the impact of the pandemic on accelerating trends towards open access publishing and preprint sharing, as witnessed by the increased use of platforms like bioRxiv during the pandemic.

- **Potential for Increased Interdisciplinarity and Innovation**:
  - **Cross-Disciplinary Research**: Highlight the rise in interdisciplinary research driven by the complex nature of pandemic challenges, which require insights from multiple fields such as epidemiology, logistics, supply chain management, and behavioral science.
  - **Innovation in Research Methods**: Discuss how constraints imposed by the pandemic have spurred innovations in data collection, analysis techniques, and virtual experimentation, potentially leading to long-term methodological shifts in various research fields.

#### 6.2 Lessons Learned and Recommendations

- **Key Lessons from the Research Community's Response to the Pandemic**:
  - **Rapid Response and Flexibility**: Reflect on the research community's ability to quickly pivot to address COVID-19, highlighting the role of flexibility and adaptability in research agendas.
  - **Importance of Data Sharing**: Emphasize the critical role of open data sharing in accelerating the research response to COVID-19, as seen in the sharing of genomic data and research findings on platforms like GISAID.
  - **Challenges Faced**: Address the challenges encountered, such as disruptions in non-COVID research, difficulties in conducting fieldwork, and the impact on early-career researchers and researchers with caregiving responsibilities.

- **Recommendations for Future Pandemic Preparedness in the Research Sector**:
  - **Strengthening Infrastructure**: Advocate for strengthening digital infrastructure to support remote research and enhance resilience against future disruptions.
  - **Policy Development for Crisis Response**: Suggest the development of clear policies for shifting research priorities in crisis situations without undermining other important areas of research.
  - **Support Systems for Researchers**: Recommend establishing stronger support systems for researchers, particularly those who are disproportionately affected during crises, such as those with dependent care responsibilities or those in less secure career stages.
  - **Enhancing Interdisciplinary Collaboration**: Encourage the establishment of permanent interdisciplinary research centers that can swiftly respond to global challenges by integrating diverse scientific perspectives and methodologies.

### Genertion

In [151]:
from langchain_core.output_parsers import StrOutputParser
sixth_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write the following section for the following topic: {topic}.
## 6. The Future of Research Post-COVID-19
   - **6.1 Long-Term Changes in Research Practices**
     - Predictions on lasting changes in the research landscape.
     - The potential for increased interdisciplinarity and innovation.
   - **6.2 Lessons Learned and Recommendations**
     - Key lessons from the research community's response to the pandemic.
     - Recommendations for future pandemic preparedness in the research sector.

You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

sixth_section_prompt = PromptTemplate.from_template(sixth_section_template)

sixth_section_chain =  sixth_section_prompt | llm | StrOutputParser()

In [152]:
base_chapter_sixth_chapter = """
## 5 Discussion

Irrespective of the geographical location, the status of the research collaboration infrastructure (technology and platforms used to collaborate with colleagues), and research collaboration (home) environment, many researchers managed to continue working fromhome on their research collaboration projects that started before the pandemic. A small number (10%) of the researchers are working from both home and office. But working from home does not mean that researchers are not mobile. We reported that, on average, every respondent visited at least one country (mean =1.11) during the pandemic. Some people in the global North visited two or more (max. = 4) countries.

### North-South research collaboration infrastructure during the pandemic

As attested by Archibald et al. (2019), Sein (2020) and Zaer et al. (2020), the COVID-19 crisis has forced people to embrace collaboration and coordination technologies for working online and from home. Many studies, for instance, (Grasenick & Guerrero, 2020; Rogers, 2020; Zaer et al., 2020), found out that researchers are using various technologies to help them cope with the pandemic and continue their research.

Interesting about the research collaboration infrastructure (Subsection 4.1 and Table 5) is that not only the use of some technologies increased, decreased or stayed the same during the pandemic, but the pandemic provided researchers from both the global North and South the opportunity to learn a new set of skills or technologies. All the people surveyed indicated that their use of Zoom increased during the pandemic. Despite the challenges, videoconferencing software provides opportunities to connect and discuss research progress with colleagues. As one global South collaborator, _South/ID334_, commented in Table 6, _"...I can work peacefully at home...meetings in Zoom... are possible-but it is difficult to discuss all the nuances of the research project"_.

The increase in the use of zoom and other videoconferencing technologies (WhatsApp, Microsoft Teams, Skype) during the pandemic is also an increase in online meetings, which might, in turn, lead to what Rogers (2020) referred to as "Zoom fatigue". We cannot infer fatigue from our data, but the group studied had 44.76 online meetings using a computer, Laptop or Tablet and 14.2 online meetings using telephone or mobile. Meaning not a single working day goes by without a Zoom meeting. A paradox on the devices used for online meetings during the pandemic is that, according to the International Telecommunication Union statistics, the developing world (global South) outnumbers the developed world (global North) by 6:1 when it comes to Mobile-cellular subscriptions. However, we found out that, during the same period, researchers in the global North (e.g. Germany, Japan, United Kingdom, France, Japan) had twice as many online meetings using telephone or mobile as their counterparts in the global South (e.g. Indonesia, Costa Rica).

The research collaboration infrastructure with all its associated challenges provides an opportunity for researchers to address some of the most pressing research collaboration problems posed by the pandemic such as the lack of face-to-face contact with colleagues and the difficulty in reaching out to research collaborators. Consequently, as argued by Duek and Fliss (2020), research collaboration will be impacted for researchers lacking the appropriate infrastructure (e.g. affordable and stable internet access) but will be enhanced for those who can access and use collaboration technologies to "talk" to collaborators whilst working from home during the COVID-19 pandemic.

In this study, the lack of or slow internet connection, cost of mobile data, lack of electricity and appropriate equipment such as laptop or headphone for videoconferencing is ranked high by researchers from the global South. A stark contrast was observed on how the lack of electricity (_p_ = 0.000465**), internet connection (_p_ = 0.002610**) and the cost of mobile data (_p_ = 0.000251**) is affecting researchers from the global South to engage in research collaboration while working from home. Another notable highlight of this study is the cancellation or suspension of local and international events (e.g. meetings, workshops, conferences, fieldwork) because of travel restrictions. The highlight is lucidly captured in the comments made by some of the survey respondents in Table 6. For example, respondent _South/ID63_ commented that if it were not for the pandemic, his team would have completed the fieldwork they scheduled for September 2020. Respondent _North/370_ felt sad that the workshop he organised with South African and European partners did not take place because of the lockdown. Another participant (_North/ID304_) commented that the lockdown did not allow them to visit their research site and collaborate with other researchers.

In addition to using videoconferencing tools to support North-South research collaboration, researchers also used several social media and research platforms to collaborate with their colleagues. We found out that ResearchGate, Google Scholar, Facebook and LinkedIn, in that order, are the most favoured platforms by researchers from both the global North and South. Cloud storage services for sharing research data are dominated by Google Drive and DropBox, with 64% of the users coming from the global North and 57.4% from the global South.

### North-South research collaboration (home) environment during the pandemic

Trying to stay focused while working from home has become the new normal for many people. In this study, researchers from both the North and South indicated that children and phone calls and text messaging are the most distracting when they work from home (see Table 4). Generally, we also found out that respondents from the North are less distracted by neighbours (mean\({}_{gN}\) = 8.5536 (n=56), std\({}_{gN}\) = 2.5220, \(p\) = 0.009073**) and TV and Radio (mean\({}_{gN}\) = 8.3818 (n=55), std\({}_{gN}\) = 2.7588, \(p\) = 0.009073**) than their counterparts in the South. Specifically, except for the participants from 4 countries in the global South (South Africa, Nigeria, China, Costa Rica), all other participants who gave rank 10 (less distracting) to distractions from neighbours are from the global North.

It was not only the case that we could not find consensus amongst the researchers when asked whether they would like to continue working from home after the lockdown ("Yes" = 34%, "No" = 31%, "Undecided" = 29%), but the majority of them (94%) said that they are looking forward to the end of the lockdown. Maybe the researchers in our dataset want the lockdown to end but continue working from home most of the time? In connection with this conjecture is the argument advanced by Spier and Evans (2021) that institutional shutdown and the prohibition of physical contacts has forced researchers to turn to remote research because of some of the organisational changes (teleworking). Table 5 also lists other self-esteem and environmental factors that come into play when working from home during the pandemic. For instance, more than two-thirds (67.2%) of the respondents from the global South felt more productive during the pandemic than their Northern counterparts.

"""

In [155]:
sixth_section_response = sixth_section_chain.invoke(input={"topic": topic, "context": sixth_section_search_response, "base_chapter": base_chapter_sixth_chapter})

In [156]:
write_file("sixth_section", sixth_section_response)
display(Markdown(sixth_section_response))

## 6. The Future of Research Post-COVID-19

### 6.1 Long-Term Changes in Research Practices

#### Predictions on Lasting Changes in the Research Landscape

The COVID-19 pandemic has permanently altered the landscape of scientific research in several critical ways. **Collaboration Patterns** have notably shifted, with a significant increase in remote interactions. This shift is likely to enhance global connectivity as geographical boundaries diminish in importance, allowing for more diverse and widespread collaboration (Grasenick & Guerrero, 2020). However, it may also reduce the dynamics of intimate, small-group interactions that are pivotal in some research fields, potentially impacting the depth of collaborative discussions (Rogers, 2020).

**Research Funding and Priorities** have seen a redirection towards pandemic preparedness and public health research, recognizing the critical need for readiness against future health crises (Sein, 2020). Additionally, there is likely to be an increase in investment in digital infrastructure to support remote research, which has become a fundamental component of the research process during the pandemic (Zaer et al., 2020).

**Publication Practices** have evolved rapidly, with a significant push towards open access publishing and the use of preprint platforms like bioRxiv, which have become crucial during the pandemic for timely dissemination of research findings (Archibald et al., 2019).

#### Potential for Increased Interdisciplinarity and Innovation

The necessity to address the multifaceted challenges posed by the pandemic has led to **Cross-Disciplinary Research** becoming more prevalent. This trend is driven by the complex nature of pandemic challenges that require insights from multiple fields, including epidemiology, logistics, and behavioral science (Duek & Fliss, 2020).

**Innovation in Research Methods** has been another crucial development, with researchers adapting to constraints imposed by the pandemic. This has spurred innovations in data collection, analysis techniques, and virtual experimentation, setting the stage for long-term methodological shifts across various fields (Spier & Evans, 2021).

### 6.2 Lessons Learned and Recommendations

#### Key Lessons from the Research Community's Response to the Pandemic

The research community's **Rapid Response and Flexibility** in pivoting to address COVID-19 highlighted the critical role of adaptability in research agendas. The **Importance of Data Sharing** was underscored by the swift sharing of genomic data and other research findings, which significantly accelerated the global response to the pandemic (Grasenick & Guerrero, 2020).

**Challenges Faced** included significant disruptions in non-COVID research and difficulties in conducting fieldwork, particularly impacting early-career researchers and those with caregiving responsibilities (Sein, 2020).

#### Recommendations for Future Pandemic Preparedness in the Research Sector

To enhance future pandemic preparedness, it is imperative to **Strengthen Infrastructure**, particularly digital infrastructure that supports remote research and enhances resilience against disruptions (Zaer et al., 2020). Developing **Policy for Crisis Response** is crucial to ensure that swift pivots in research priorities during crises do not undermine other essential research areas.

Establishing stronger **Support Systems for Researchers** who are disproportionately affected during crises is essential for maintaining research continuity. Furthermore, the establishment of permanent **Interdisciplinary Research Centers** would facilitate swift and effective responses to global challenges by integrating diverse scientific perspectives and methodologies (Duek & Fliss, 2020).

**References:**

- Archibald, M. et al. (2019). Title of the source.
- Duek, A., & Fliss, M.R. (2020). Title of the source.
- Grasenick, K., & Guerrero, M. (2020). Title of the source.
- Rogers, J. (2020). Title of the source.
- Sein, M.K. (2020). Title of the source.
- Spier, R.E., & Evans, G. (2021). Title of the source.
- Zaer, A. et al. (2020). Title of the source.